In [1]:
import numpy as np
from sklearn import datasets
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import math

In [2]:
iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
x=iris.data

In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [6]:
def entropy(arr, total):
    sum=0    # arr is an array of count of zeroes, ones and twos in  case of iris dataset!
    for i in range(len(arr)):
        if(arr[i]==0):
            continue
        sum-=(arr[i]/total)*math.log2(arr[i]/total)
    return sum

In [7]:
class TreeNode:

    def __init__(self, data,CurrLevel,yValues,gr=-10000000000,entropy=-1, message="",splitIndex=-1,splitValue=0,classValue=None):
     
        self.left = None
        self.right = None
        self.dataArray = data
        self.level=CurrLevel
        self.gainRatio=gr
        self.entropy=entropy
        self.message=message
        self.yValues=yValues
        self.splitIndex=splitIndex
        self.splitValue=splitValue
        self.classValue=classValue
        

In [8]:
mean=np.zeros(x.shape[1])
mean=mean.reshape(1,4)
y.shape

(150,)

In [9]:
def infoGain(arr1,arr2,arr3):
    t=arr1.sum()
    oldEntropy=entropy(arr1,arr1.sum())
    newEntropy=(arr2.sum()*entropy(arr2,arr2.sum())+arr3.sum()*entropy(arr3,arr3.sum()))/t
    return oldEntropy-newEntropy

In [10]:
def gainRatio(infoGain, splits=2):
    return infoGain/splits

In [12]:
visited=[False, False,False, False]
# A VISITED ARRAY MAINTAINED TO KEEP A CHECK ON WHAT ALL FEATURES HAVE BEEN TRAVERSED
def recursion_Tree_Formation(arr, level, mean, visited, No_of_visited, xoutput, youtput, root):
    e=entropy(arr,arr.sum())
    
    root.data=arr
    root.entropy=e
    
    
    if e==0:
       # Leaf Node
       # Formation of pure Node
        classValue=-1
        for i in range(3):
            if(arr[i]!=0):
                classValue=i
                break
        root.classValue=classValue
        root.left=None
        root.right=None
        root.message= "Pure Node Formed"
        
        return
    
    if(No_of_visited==4):  
        # All features traversed
        # Leaf Node
        freq=-1
        majorityClass=-1
        for i in range(3):
            if(arr[i]>freq):
                freq=arr[i]
                majorityClass=i
        root.classValue=majorityClass
        root.left=None
        root.right=None
        root.message= "All features traversed"
        
        return
    
    
    #  otherwise splitting on feature.. with gain ratio..
    for i in range(4):
        mean[0,i]=xoutput[:,i].mean()
       
    # initially some value of gain ratio is considered
    gain_ratio=-10000000000
    index=0
    # A count of the each class value is maintained for calculating entropy
    LEFT =np.zeros(3)   
    RIGHT=np.zeros(3)      
    # includes rows when splitting is done
    LeftOutput=np.zeros((1,4))    
    RightOutput=np.zeros((1,4))
# traversing every column (feature) to identify the feature on the basis of which splitting is to be done
# feature with max gain ratio will be selected
    for i in range(4):
        
        if(visited[i]==False):
            left=np.zeros(3)
            right=np.zeros(3)
            temp=xoutput[:,i] # the current column 
            for j in range(xoutput.shape[0]):
                if temp[j]<mean[0,i]:
                    left[int(youtput[j])]+=1
                else:
                    right[int(youtput[j])]+=1
            informationGain=infoGain(arr,left,right)
            gr=gainRatio(informationGain,2)
            
            if gr>gain_ratio:
                gain_ratio=gr
                index=i
                LEFT=left
                RIGHT=right
    visited[index]=True
    
    # Arrays resulted when splitting is done- LeftOutput, RightOutput
    # Arrays keeping a record of corresponding y values- yOutputLeft, yOutputRight
    LeftOutput  = np.vstack((LeftOutput, xoutput[xoutput[:,index] < mean[0,index]])) 
    LeftOutput  = LeftOutput[1:,:]
    yOutputLeft = np.zeros(LeftOutput.shape[0],)
    
    RightOutput = np.vstack((RightOutput, xoutput[xoutput[:,index] >= mean[0,index]])) 
    RightOutput = RightOutput[1:,:]
    yOutputRight=np.zeros(RightOutput.shape[0])
    
    l=0
    r=0
    for i in range(xoutput.shape[0]):
        if(xoutput[i,index]<mean[0,index]):
            yOutputLeft[l]=youtput[i]
            l+=1
        else:
            yOutputRight[r]=youtput[i]
            r+=1
            
    root.gainRatio=gain_ratio
    
    leftNode=TreeNode(LEFT,level+1,yOutputLeft)  
    rightNode=TreeNode(RIGHT,level+1,yOutputRight)

    
    root.left=leftNode
    root.right=rightNode
    root.splitIndex=index
    root.splitValue=mean[0,index]
    # Recursive function called for complete formation of the tree
 
    recursion_Tree_Formation(LEFT, level+1, mean, visited.copy(), No_of_visited+1, LeftOutput, yOutputLeft , root.left)

    recursion_Tree_Formation(RIGHT, level+1, mean, visited.copy(), No_of_visited+1, RightOutput, yOutputRight , root.right)
        
        

In [13]:
arr=np.arange(3)
arr[0]=arr[1]=arr[2]=50
arr

array([50, 50, 50])

In [14]:
entropy(arr,arr.sum())

1.584962500721156

In [15]:
mainRoot=TreeNode(arr,0,arr)


In [16]:
recursion_Tree_Formation(arr,0,mean,visited,0,x,y,mainRoot)

In [17]:
def printTree(root):
    if(root==None):
        return
    
    arr=root.data
    e=root.entropy
    print("Level = ", root.level)
    print("Count of 0 = ",arr[0])
    print("Count of 1 = ",arr[1]) 
    print("Count of 2 = ",arr[2])
    print("Current entropy is= ", e)
    
    if (root.left==None and root.right==None):
        print(root.message)
    else:   
        print("Splitting on feature ",iris.feature_names[root.splitIndex]," with gain ratio ",root.gainRatio)
        
    print()
    print()
    printTree(root.left)
    printTree(root.right)
    

In [18]:
printTree(mainRoot)

Level =  0
Count of 0 =  50
Count of 1 =  50
Count of 2 =  50
Current entropy is=  1.584962500721156
Splitting on feature  petal length (cm)  with gain ratio  0.3816478758393404


Level =  1
Count of 0 =  50.0
Count of 1 =  7.0
Count of 2 =  0.0
Current entropy is=  0.5373760853377336
Splitting on feature  petal width (cm)  with gain ratio  0.12992321343539348


Level =  2
Count of 0 =  41.0
Count of 1 =  0.0
Count of 2 =  0.0
Current entropy is=  0.0
Pure Node Formed


Level =  2
Count of 0 =  9.0
Count of 1 =  7.0
Count of 2 =  0.0
Current entropy is=  0.9886994082884974
Splitting on feature  sepal width (cm)  with gain ratio  0.4943497041442487


Level =  3
Count of 0 =  0.0
Count of 1 =  7.0
Count of 2 =  0.0
Current entropy is=  0.0
Pure Node Formed


Level =  3
Count of 0 =  9.0
Count of 1 =  0.0
Count of 2 =  0.0
Current entropy is=  0.0
Pure Node Formed


Level =  1
Count of 0 =  0.0
Count of 1 =  43.0
Count of 2 =  50.0
Current entropy is=  0.9959094138937685
Splitting on feat

In [19]:
def fit(mainRoot):
    TreeRoot=mainRoot
    return TreeNode

In [20]:
def predictSingle(x_tuple,root):
    if(root.left == None and root.right==None):
        predictedClass=root.classValue
   
        return predictedClass
    
    index=root.splitIndex
    if(x_tuple[index]<root.splitValue):
        root=root.left
    else:
        root=root.right
    return predictSingle(x_tuple,root)    

In [21]:
def predict(x,root):
    y_pred=np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        pred=predictSingle(x[i],root)
        y_pred[i]=(pred)
    return y_pred

In [22]:
y_pred=predict(x,mainRoot)

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
# converting list to numpy
y_pred=np.asarray(y_pred)

In [25]:
accuracy_score(y_pred,y)

0.96